# Day 10: Monitoring Station
You fly into the asteroid belt and reach the Ceres monitoring station. The Elves here have an emergency: they're having trouble tracking all of the asteroids and can't be sure they're safe.

The Elves would like to build a new monitoring station in a nearby area of space; they hand you a map of all of the asteroids in that region (your puzzle input).

The map indicates whether each position is empty (.) or contains an asteroid (#). The asteroids are much smaller than they appear on the map, and every asteroid is exactly in the center of its marked position. The asteroids can be described with X,Y coordinates where X is the distance from the left edge and Y is the distance from the top edge (so the top-left corner is 0,0 and the position immediately to its right is 1,0).

Your job is to figure out which asteroid would be the best place to build a new monitoring station. A monitoring station can detect any asteroid to which it has direct line of sight - that is, there cannot be another asteroid exactly between them. This line of sight can be at any angle, not just lines aligned to the grid or diagonally. The best location is the asteroid that can detect the largest number of other asteroids.

For example, consider the following map:

<pre>
.#..#
.....
#####
....#
...<b>#</b>#
</pre>

The best location for a new monitoring station on this map is the highlighted asteroid at 3,4 because it can detect 8 asteroids, more than any other location. (The only asteroid it cannot detect is the one at 1,0; its view of this asteroid is blocked by the asteroid at 2,2.) All other asteroids are worse locations; they can detect 7 or fewer other asteroids. Here is the number of other asteroids a monitoring station on each asteroid could detect:

```
.7..7
.....
67775
....7
...87
```

Here is an asteroid (#) and some examples of the ways its line of sight might be blocked. If there were another asteroid at the location of a capital letter, the locations marked with the corresponding lowercase letter would be blocked and could not be detected:

```
#.........
...A......
...B..a...
.EDCG....a
..F.c.b...
.....c....
..efd.c.gb
.......c..
....f...c.
...e..d..c
```

Here are some larger examples:

Best is 5,8 with 33 other asteroids detected:

```
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####
```

Best is 1,2 with 35 other asteroids detected:

```
#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.
```

Best is 6,3 with 41 other asteroids detected:

```
.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
```

Best is 11,13 with 210 other asteroids detected:

```
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
```

Find the best location for a new monitoring station. How many other asteroids can be detected from that location?

In [10]:
import enum

class Space(enum.Enum):
    EMPTY = '.'
    ASTEROID = '#'

def load_asteroid_map(s):
    return [[Space.ASTEROID if c == '#' else Space.EMPTY for c in line] for line in s.strip().split('\n')]

In [2]:
def asteroid_locations(map):
    for y, row in enumerate(map):
        for x, thing in enumerate(row):
            if thing is Space.ASTEROID:
                yield (x, y)

In [3]:
def vector(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return (x2 - x1, y2 - y1)

In [83]:
def sign(v):
    return 1 if v > 0 else -1

In [85]:
def asteroids_visible(asteroids, point):
    x, y = point
    visible = set()
    for asteroid_location in asteroids:
        if asteroid_location == point:
            # We can't monitor ourself
            continue
        x1, y1 = vector(point, asteroid_location)
        x1_sign, y1_sign = sign(x1), sign(y1)
        for seen_asteroid in visible:
            x2, y2 = vector(point, seen_asteroid)
            # x1     x2
            # --  =  --
            # y1     y2
            # We want these to be equal, but also the signs to match;
            # for instance (-1, 1) doesn't block vision of (1, -1) from (0, 0)
            # even though they are all co-linear. We check both because otherwise
            # if x1, x2 == 0 for instance, we don't want y1 = -1 to block y2 = 1
            if x1 * y2 == x2 * y1 and (x1_sign, y1_sign) == (sign(x2), sign(y2)):
                break
        else:
            visible.add(asteroid_location)
    return visible

In [63]:
def best_monitoring_point(map):
    asteroids = list(asteroid_locations(map))
    best_point = max(asteroids, key=lambda point: len(asteroids_visible(asteroids, point)))
    visible_asteroids = asteroids_visible(asteroids, best_point)
    return best_point, len(visible_asteroids)

In [35]:
test_map = load_asteroid_map('''
.#..#
.....
#####
....#
...##
''')

In [36]:
test_asteroids = list(asteroid_locations(test_map))

In [51]:
num_asteroids_visible(test_asteroids, (1, 0))

Saw from (1, 0): {(1, 2), (3, 2), (4, 4), (2, 2), (4, 2), (0, 2), (4, 0)}


7

In [46]:
def debug_map(map, point):
    visible_points = asteroids_visible(list(asteroid_locations(map)), point)
    for y, row in enumerate(map):
        row_str = ''
        for x, thing in enumerate(row):
            if (x, y) in visible_points:
                row_str += '+'
            elif (x, y) == point:
                row_str += '#'
            elif thing is Space.EMPTY:
                row_str += '.'
            else:
                row_str += '-'
        print(row_str)

In [52]:
debug_map(test_map, (1, 0))

.#..+
.....
+++++
....-
...-+


In [64]:
best_monitoring_point(test_map)

((3, 4), 8)

In [55]:
test_map_large = load_asteroid_map('''
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
''')

In [59]:
len(asteroids_visible(list(asteroid_locations(test_map_large)), (11, 13)))

154

In [60]:
# Best is 5,8 with 33 other asteroids detected:
test_map_1 = load_asteroid_map('''
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####
''')

In [65]:
best_monitoring_point(test_map_1)

((0, 1), 32)

In [62]:
debug_map(test_map_1, (0, 1))

......+.+.
#..+.-....
..+++++++.
.+.+.+-+..
.+..+.....
..-....+.+
+..+....+.
.+-.+..+--
-+...+..+.
.+....++++


In [68]:
def num_asteroids_visible(map, point):
    return len(asteroids_visible(list(asteroid_locations(map)), point))

In [87]:
num_asteroids_visible(test_map_1, (5, 8))

33

In [86]:
debug_map(test_map_1, (5, 8))

......+.+.
+..+.+....
..+++-+++.
.+.+.-++..
.+..+.....
..-....+.+
+..-....+.
.++.-..+++
+-...#..+.
.+....++++


In [89]:
num_asteroids_visible(test_map_large, (11, 13))

210

In [90]:
from tools import get_data

In [91]:
data = get_data(10)

In [92]:
best_monitoring_point(load_asteroid_map(data))

((37, 25), 309)

### Part Two
Once you give them the coordinates, the Elves quickly deploy an Instant Monitoring Station to the location and discover the worst: there are simply too many asteroids.

The only solution is complete vaporization by giant laser.

Fortunately, in addition to an asteroid scanner, the new monitoring station also comes equipped with a giant rotating laser perfect for vaporizing asteroids. The laser starts by pointing up and always rotates clockwise, vaporizing any asteroid it hits.

If multiple asteroids are exactly in line with the station, the laser only has enough power to vaporize one of them before continuing its rotation. In other words, the same asteroids that can be detected can be vaporized, but if vaporizing one asteroid makes another one detectable, the newly-detected asteroid won't be vaporized until the laser has returned to the same position by rotating a full 360 degrees.

For example, consider the following map, where the asteroid with the new monitoring station (and laser) is marked X:

```
.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....X...###..
..#.#.....#....##
```

The first nine asteroids to get vaporized, in order, would be:

```
.#....###24...#..
##...##.13#67..9#
##...#...5.8####.
..#.....X...###..
..#.#.....#....##
```
Note that some asteroids (the ones behind the asteroids marked 1, 5, and 7) won't have a chance to be vaporized until the next full rotation. The laser continues rotating; the next nine to be vaporized are:

```
.#....###.....#..
##...##...#.....#
##...#......1234.
..#.....X...5##..
..#.9.....8....76
```

The next nine to be vaporized are then:

```
.8....###.....#..
56...9#...#.....#
34...7...........
..2.....X....##..
..1..............
```

Finally, the laser completes its first full rotation (1 through 3), a second rotation (4 through 8), and vaporizes the last asteroid (9) partway through its third rotation:

```
......234.....6..
......1...5.....7
.................
........X....89..
.................
```
In the large example above (the one with the best monitoring station location at 11,13):

- The 1st asteroid to be vaporized is at 11,12.
- The 2nd asteroid to be vaporized is at 12,1.
- The 3rd asteroid to be vaporized is at 12,2.
- The 10th asteroid to be vaporized is at 12,8.
- The 20th asteroid to be vaporized is at 16,0.
- The 50th asteroid to be vaporized is at 16,9.
- The 100th asteroid to be vaporized is at 10,16.
- The 199th asteroid to be vaporized is at 9,6.
- The 200th asteroid to be vaporized is at 8,2.
- The 201st asteroid to be vaporized is at 10,9.
- The 299th and final asteroid to be vaporized is at 11,1.

The Elves are placing bets on which will be the 200th asteroid to be vaporized. Win the bet by determining which asteroid that will be; what do you get if you multiply its X coordinate by 100 and then add its Y coordinate? (For example, 8,2 becomes 802.)

In [93]:
def sort_magnitude(vector):
    x, y = vector
    return x * x + y * y

In [95]:
import collections

In [166]:
def vector_paths(asteroids, point):
    x, y = point
    visible = set()
    vector_paths = collections.defaultdict(list)
    for asteroid_location in asteroids:
        if asteroid_location == point:
            # We can't monitor ourself
            continue
        x1, y1 = vector(point, asteroid_location)
        x1_sign, y1_sign = sign(x1), sign(y1)
        for seen_asteroid in visible:
            x2, y2 = vector(point, seen_asteroid)
            # x1     x2
            # --  =  --
            # y1     y2
            # We want these to be equal, but also the signs to match;
            # for instance (-1, 1) doesn't block vision of (1, -1) from (0, 0)
            # even though they are all co-linear. We check both because otherwise
            # if x1, x2 == 0 for instance, we don't want y1 = -1 to block y2 = 1
            if x1 * y2 == x2 * y1 and (x1_sign, y1_sign) == (sign(x2), sign(y2)):
                vector_paths[seen_asteroid].append(asteroid_location)
                break
        else:
            visible.add(asteroid_location)
            vector_paths[asteroid_location].append(asteroid_location)
    return [
        sorted(path, key=lambda asteroid: sort_magnitude(vector(point, asteroid)))
        for path in vector_paths.values()
    ]

In [168]:
import math

def tau(vector):
    x, y = vector
    if y >= 0:
        return math.atan2(y, x)
        # For x < 0, atan2 flips to -pi at (0, -1), and then goes down to 0.
        # This is right if we basically mod with 2pi
    return (2 * math.pi) + math.atan2(y, x)

def vtau(vector):
    x, y = vector
    # We want 0 to start at (0, 1) rather than (1, 0) and to go clockwise
    # rather than counterclockwise; atan2 takes in (y, x), but if we invert them
    # we reflect over x = y which gives us what we want.
    # Also, in this vector space, y is inverted, because "up" goes down to zero.
    return tau((-y, x))

In [169]:
vtau((-0.001, 1))

3.1425926532564596

In [170]:
vtau((0, 1))

3.141592653589793

In [171]:
vtau((1, 0))

1.5707963267948966

In [172]:
vtau((0, -1))

0.0

In [173]:
vtau((-0.0001, -1))

6.2830853071799195

In [174]:
vtau((-1, 0))

4.71238898038469

In [184]:
def destroy_in_order_with_laser(asteroids, point):
    paths = vector_paths(asteroids, point)
    paths = sorted(paths, key=lambda path: vtau(vector(point, path[0])))
    path_iters = [iter(path) for path in paths]
    while path_iters:
        empty_path_iters = set()
        for path_iter in path_iters:
            try:
                yield next(path_iter)
            except StopIteration:
                empty_path_iters.add(path_iter)
        path_iters = [path_iter for path_iter in path_iters if path_iter not in empty_path_iters]

In [156]:
def laser(map, point):
    asteroids = list(asteroid_locations(map))
    return list(
        destroy_in_order_with_laser(asteroids, point)
    )

In [177]:
def debug_vector_paths(map, point):
    asteroids = list(asteroid_locations(map))
    paths = vector_paths(asteroids, point)
    paths = sorted(paths, key=lambda path: vtau(vector(point, path[0])))
    print("Sorted vector paths:")
    for path in paths:
        print(vtau(vector(point, path[0])), path)

In [178]:
debug_vector_paths(test_map_large, (11, 13))

Sorted vector paths:
0.0 [(11, 12), (11, 11), (11, 10), (11, 9), (11, 8), (11, 7), (11, 6), (11, 5), (11, 4), (11, 3), (11, 2), (11, 1)]
0.08314123188844123 [(12, 1)]
0.09065988720074511 [(12, 2)]
0.11065722117389565 [(12, 4)]
0.12435499454676144 [(12, 5)]
0.14189705460416394 [(12, 6)]
0.15264932839526515 [(13, 0)]
0.16514867741462683 [(12, 7), (13, 1)]
0.17985349979247828 [(13, 2)]
0.19739555984988075 [(12, 8), (13, 3)]
0.22679884805388587 [(14, 0)]
0.24497866312686414 [(13, 5), (14, 1)]
0.2662520491509254 [(14, 2)]
0.27829965900511133 [(13, 6)]
0.2914567944778671 [(14, 3)]
0.2984989315861793 [(15, 0)]
0.3217505543966422 [(14, 4)]
0.348771003583907 [(15, 2)]
0.35877067027057225 [(14, 5)]
0.3671738338182192 [(16, 0)]
0.3805063771123649 [(13, 8), (15, 3)]
0.40489178628508343 [(14, 6)]
0.4182243295792291 [(15, 4)]
0.4266274931268761 [(16, 2)]
0.43240777557053783 [(17, 0)]
0.4636476090008061 [(14, 7), (15, 5), (16, 3), (17, 1)]
0.49394136891958124 [(18, 0)]
0.4993467216801301 [(17, 2)]
0.

In [185]:
destroy_order = laser(test_map_large, (11, 13))

In [186]:
destroy_order[0]

(11, 12)

In [187]:
assert destroy_order[0] == (11, 12)

In [188]:
The 1st asteroid to be vaporized is at 11,12.
The 2nd asteroid to be vaporized is at 12,1.
The 3rd asteroid to be vaporized is at 12,2.
The 10th asteroid to be vaporized is at 12,8.
The 20th asteroid to be vaporized is at 16,0.
The 50th asteroid to be vaporized is at 16,9.
The 100th asteroid to be vaporized is at 10,16.
The 199th asteroid to be vaporized is at 9,6.
The 200th asteroid to be vaporized is at 8,2.
The 201st asteroid to be vaporized is at 10,9.
The 299th and final asteroid to be vaporized is at 11,1.

[(11, 12),
 (12, 1),
 (12, 2),
 (12, 4),
 (12, 5),
 (12, 6),
 (13, 0),
 (12, 7),
 (13, 2),
 (12, 8),
 (14, 0),
 (13, 5),
 (14, 2),
 (13, 6),
 (14, 3),
 (15, 0),
 (14, 4),
 (15, 2),
 (14, 5),
 (16, 0),
 (13, 8),
 (14, 6),
 (15, 4),
 (16, 2),
 (17, 0),
 (14, 7),
 (18, 0),
 (17, 2),
 (16, 4),
 (15, 6),
 (18, 1),
 (14, 8),
 (19, 0),
 (16, 5),
 (13, 10),
 (18, 3),
 (16, 6),
 (19, 2),
 (14, 9),
 (18, 4),
 (15, 8),
 (16, 7),
 (17, 6),
 (18, 5),
 (19, 4),
 (12, 12),
 (19, 6),
 (18, 7),
 (17, 8),
 (16, 9),
 (15, 10),
 (18, 8),
 (17, 9),
 (19, 8),
 (16, 10),
 (13, 12),
 (18, 10),
 (16, 11),
 (19, 10),
 (14, 12),
 (15, 12),
 (17, 12),
 (18, 12),
 (19, 12),
 (12, 13),
 (19, 14),
 (18, 14),
 (17, 14),
 (16, 14),
 (15, 14),
 (18, 15),
 (14, 14),
 (19, 16),
 (16, 15),
 (18, 16),
 (13, 14),
 (18, 17),
 (16, 16),
 (19, 18),
 (14, 15),
 (18, 18),
 (15, 16),
 (17, 18),
 (18, 19),
 (13, 15),
 (15, 18),
 (14, 17),
 (15, 19),
 (14, 18),
 (12, 15),
 (13, 18),
 (12, 16),
 (12, 17),
 (12, 18),
 (12, 19),
 (11,

In [189]:
assert destroy_order[0] == (11, 12)
assert destroy_order[1] == (12, 1)
assert destroy_order[2] == (12, 2)
assert destroy_order[9] == (12, 8)
assert destroy_order[19] == (16, 0)
assert destroy_order[49] == (16, 9)
assert destroy_order[99] == (10, 16)
assert destroy_order[198] == (9, 6)
assert destroy_order[199] == (8, 2)
assert destroy_order[200] == (10, 9)
assert destroy_order[298] == (11, 1)

In [ ]:
laser(load_asteroid_map(data), (11, 13))